In [28]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

In [2]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

C:\Users\power\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\power\.cache\huggingface\hub\models--nlptown--bert-base-multilingual-uncased-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

C:\Users\power\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

C:\Users\power\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
tokens = tokenizer.encode('I am learning, it is good', return_tensors = 'pt')

In [5]:
tokens[0]

tensor([  101,   151, 10345, 19956,   117, 10197, 10127, 12050,   102])

In [6]:
tokenizer.decode(tokens[0])

'[CLS] i am learning, it is good [SEP]'

In [7]:
result = model(tokens)

In [8]:
result

SequenceClassifierOutput(loss=None, logits=tensor([[-2.4137, -1.7810,  0.7707,  1.7757,  1.2946]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [10]:
result.logits

tensor([[-2.4137, -1.7810,  0.7707,  1.7757,  1.2946]],
       grad_fn=<AddmmBackward0>)

In [11]:
torch.argmax(result.logits)

tensor(3)

In [12]:
int(torch.argmax(result.logits)) + 1

4

In [17]:
tokens2 = tokenizer.encode('I hate this app, i will delete it', return_tensors = 'pt')

In [18]:
tokens2

tensor([[  101,   151, 39487, 10372, 35821,   117,   151, 11229, 26962, 10218,
         10197,   102]])

In [19]:
result2 = model(tokens2)

In [22]:
result2.logits

tensor([[ 3.4595,  1.8680, -0.0303, -2.1521, -2.4029]],
       grad_fn=<AddmmBackward0>)

In [24]:
torch.argmax(result2.logits)

tensor(0)

In [25]:
int(torch.argmax(result2.logits)) + 1

1

In [26]:
r = requests.get('https://www.yelp.com/biz/mejico-sydney-2')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*comment.*')
results = soup.find_all('p', {'class' : regex})
reviews = [result.text for result in results]

In [27]:
reviews[0]

"Seated without a booking on a super busy Saturday night. Lovely, warm, and Theo right hostess also looked after our table and went out of her way to give detailed ingredients in every dish to avoid allergies for one of us. And the food was great! Guacamole made right at our table, everything prepared with our allergies in mind, and great dish recommendations. We'd been visiting Sydney for about a week from Melbourne, and this was by far our best dining experience. I'd definitely return here in the future."

In [29]:
df = pd.DataFrame(np.array(reviews), columns=['reviews'])

In [30]:
df.head()

,reviews
0,Seated without a booking on a super busy Satur...
1,The food was decent not great.. We had the gu...
2,"Food was okay, guacamole was below average. Se..."
3,The food and service here was really good. It...
4,Visiting from Texas and decided to give this r...


In [33]:
def sentiment_reviews(review):
    token = tokenizer.encode(review, return_tensors = 'pt')
    result = model(token)
    return int(torch.argmax(result.logits)) + 1

In [34]:
df['sentiment_score'] = df['reviews'].apply(lambda x : sentiment_reviews(x[:512]))

In [36]:
df

,reviews,sentiment_score
0,Seated without a booking on a super busy Satur...,5
1,The food was decent not great.. We had the gu...,2
2,"Food was okay, guacamole was below average. Se...",2
3,The food and service here was really good. It...,5
4,Visiting from Texas and decided to give this r...,5
5,Don't come here expecting legit Mexican food b...,3
6,Out of all the restaurants that I tried in Syd...,5
7,"Great atmosphere, attentive service, solid mar...",3
8,We came here on a Thursday night @ 5pm and by ...,4
9,The food is fresh and tasty. The scallop cevi...,4
